# Data Aggregation for Dashboard

This notebook contains the data aggregation code to prepare data files for the dashboard. You can run this notebook to see how Dask is used with a Saturn cluster for data processing, but the files generated here will not be used by any of the examples. The dashboard uses pre-aggregated files from Saturn's public S3 bucket.

In [ ]:
import os

DATA_PATH = 'data'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

In [ ]:
import s3fs
import dask.dataframe as dd
import numpy as np

import hvplot.dask, hvplot.pandas

fs = s3fs.S3FileSystem(anon=True)

# Launch Dask cluster

We will need to do some data processing that exceeds the capacity of our JupyterLab client. To monitor the status of your cluster, visit the "Logs" page. 

In [ ]:
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(n_workers=n_workers, scheduler_size='medium', worker_size='large', nthreads=2)
client = Client(cluster)
cluster

If you initialized your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster from the "Dask" page in Saturn if you want to get a head start!

In [ ]:
client.wait_for_workers(n_workers=n_workers)

# Load data

Setup a function to load files with Dask. Cleanup some column names and parse data types correctly.

In [ ]:
usecols = ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
           'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount',
           'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']

def read_taxi_csv(files):
    ddf = dd.read_csv(files, 
                      assume_missing=True,
                      parse_dates=[1, 2], 
                      usecols=usecols, 
                      storage_options={'anon': True})
    # grab the columns we need and rename
    ddf = ddf[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'PULocationID', 'DOLocationID',
               'passenger_count', 'trip_distance', 'payment_type', 'tip_amount', 'fare_amount']]
    ddf.columns = ['pickup_datetime', 'dropoff_datetime', 'pickup_taxizone_id', 'dropoff_taxizone_id',
                   'passenger_count', 'trip_distance', 'payment_type', 'tip_amount', 'fare_amount']
    return ddf

Get a listing of files from the public S3 bucket

In [ ]:
files = [f's3://{x}' for x in fs.glob('s3://nyc-tlc/trip data/yellow_tripdata_201*.csv')
         if '2017' in x or '2018' in x or '2019' in x]
len(files), files[:2]

In [ ]:
ddf = read_taxi_csv(files[:5])  # only load first 5 months of data

<br>

We are loading a small sample for this exercise, but if you want to use the full data and replicate the aggregated data hosted on Saturn's bucket, you will need to use a larger cluster. Here is a sample cluster configuration you can use, but you can play around with sizes and see how performance changes!

```python
cluster = SaturnCluster(
    n_workers=10, 
    scheduler_size='xlarge',
    worker_size='8xlarge', 
    nthreads=32,
)
```

You will have to run `cluster.reset(...)` if the cluster has already been configured. Run the following to see what sizes are available:

```python
from dask_saturn.core import describe_sizes
describe_sizes()
```

In [ ]:
# load all 3 years of data
# ddf = read_taxi_csv(files)

# Aggregated files for Dashboard

Create several CSV file to use for visualization in the dashboard. Note that each of these perform some Dask dataframe operations, then call `compute()` to pull down a pandas dataframe, and then write that dataframe to a CSV file.

## Augment data

We'll distill some features out of the datetime component of the data. This is similar to the feature engineering that is done in other places in this demo, but we'll only create the features that'll be most useful in the visuals. 

In [ ]:
ddf["pickup_hour"] = ddf.pickup_datetime.dt.hour
ddf["dropoff_hour"] = ddf.dropoff_datetime.dt.hour
ddf["pickup_weekday"] = ddf.pickup_datetime.dt.weekday
ddf["dropoff_weekday"] = ddf.dropoff_datetime.dt.weekday
ddf["percent_tip"] = (ddf["tip_amount"] / ddf["fare_amount"]).replace([np.inf, -np.inf], np.nan) * 100

We'll take out the extreme high values since they disrupt the mean

In [ ]:
ddf["percent_tip"] = ddf["percent_tip"].apply(lambda x: np.nan if x > 1000 else x)

Notice that all of the above cells execute pretty much instantly. This is because of Dask's [lazy evaluation](https://tutorial.dask.org/01x_lazy.html). Calling `persist()` below tells Dask to run all the operations and keep the results in memory for faster computation. This cell takes some time to run because Dask needs to first parse all the CSV files.

In [ ]:
%%time
ddf = ddf.persist()
_ = wait(ddf)

## Timeseries datasets

We'll resample to an hourly timestep so that we don't have to pass around so much data later on.

In [ ]:
tip_ddf = ddf[["pickup_datetime", "percent_tip"]].set_index("pickup_datetime").dropna()
tips = tip_ddf.resample('1H').mean().compute()

tips.to_csv(f"{DATA_PATH}/pickup_average_percent_tip_timeseries.csv")

In [ ]:
fare_ddf = ddf[["pickup_datetime", "fare_amount"]].set_index("pickup_datetime").dropna()
fare = fare_ddf.resample('1H').mean().compute()

fare.to_csv(f"{DATA_PATH}/pickup_average_fare_timeseries.csv")

## Aggregate datasets

Since our data is rather large and will mostly be viewed in grouped aggregates, we can do some aggregation now and save it off for use in plots later. 

In [ ]:
for value in ["pickup", "dropoff"]:
    data = (ddf
            .groupby([
                f"{value}_taxizone_id", 
                f"{value}_hour",  
                f"{value}_weekday",
            ])
            .agg({
                "fare_amount": ["mean", "count", "sum"],
                "trip_distance": ["mean"],
                "percent_tip": ["mean"],
            })
            .compute()
           )
    data.columns = data.columns.to_flat_index()
    data = data.rename({
        ("fare_amount", "mean"): "average_fare",
        ("fare_amount", "count"): "total_rides",
        ("fare_amount", "sum"): "total_fare",
        ("trip_distance", "mean"): "average_trip_distance",
        ("percent_tip", "mean"): "average_percent_tip",
        
    }, axis=1).reset_index(level=[1, 2])
    data.to_csv(f"{DATA_PATH}/{value}_grouped_by_zone_and_time.csv")

grouped_zone_and_time = data

In [ ]:
for value in ["pickup", "dropoff"]:
    data = (ddf
            .groupby([
                f"{value}_taxizone_id", 
            ])
            .agg({
                "fare_amount": ["mean", "count", "sum"],
                "trip_distance": ["mean"],
                "percent_tip": ["mean"],
            })
            .compute()
           )
    data.columns = data.columns.to_flat_index()
    data = data.rename({
        ("fare_amount", "mean"): "average_fare",
        ("fare_amount", "count"): "total_rides",
        ("fare_amount", "sum"): "total_fare",
        ("trip_distance", "mean"): "average_trip_distance",
        ("percent_tip", "mean"): "average_percent_tip",
        
    }, axis=1)
    data.to_csv(f"{DATA_PATH}/{value}_grouped_by_zone.csv")

grouped_zone = data

In [ ]:
value = "pickup"
data = (ddf
        .groupby([
            f"{value}_hour", 
            f"{value}_weekday"
        ])
        .agg({
            "fare_amount": ["mean", "count", "sum"],
            "trip_distance": ["mean"],
            "percent_tip": ["mean"],
        })
        .compute()
       )
data.columns = data.columns.to_flat_index()
data = data.rename({
    ("fare_amount", "mean"): "average_fare",
    ("fare_amount", "count"): "total_rides",
    ("fare_amount", "sum"): "total_fare",
    ("trip_distance", "mean"): "average_trip_distance",
    ("percent_tip", "mean"): "average_percent_tip",

}, axis=1)

data.to_csv(f"{DATA_PATH}/{value}_grouped_by_time.csv")
grouped_time = data

## Get shape files for dashboard

The shape files are stored in a zip on the public S3. Here we pull it down, unzip it, then place the files on our S3.

In [ ]:
import zipfile
with fs.open('s3://nyc-tlc/misc/taxi_zones.zip') as f:
    with zipfile.ZipFile(f) as zip_ref:
        zip_ref.extractall(f'{DATA_PATH}/taxi_zones')

## Examples

To make use of the new datasets we can visualize all the data at once using a grouped heatmap

In [ ]:
grouped_zone_and_time.hvplot.heatmap(
    x="dropoff_weekday", 
    y="dropoff_hour", 
    C="average_percent_tip",
    groupby="dropoff_taxizone_id", 
    responsive=True, min_height=600, cmap="viridis", clim=(0, 20),
    colorbar=False,
)

This dataset that is only grouped by zone can be paired with other information such as geography.

In [ ]:
import geopandas as gpd

zones = gpd.read_file(f'{DATA_PATH}/taxi_zones/taxi_zones.shp').to_crs('epsg:4326')
joined = zones.join(grouped_zone, on="LocationID")

joined.hvplot(x="longitude", y="latitude", c="average_fare", 
              geo=True, tiles="CartoLight", cmap="fire", alpha=0.5,
              hover_cols=["zone", "borough"], 
              title="Average fare by dropoff location",
              height=600, width=800, clim=(0, 100))